In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

In [58]:
!listdir

/bin/bash: listdir: command not found


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
print("./drive/My Drive/")
print("./open?id=1HTKUCcidMJWC6rVt7COZ9zVeN97Xikw1")
print("./content/drive")

./drive/My Drive/
./open?id=1HTKUCcidMJWC6rVt7COZ9zVeN97Xikw1
./content/drive


In [0]:
!cd "./drive/My Drive/Images"

In [63]:
image_dataset=("./drive/My Drive/Images")
!listdir image_dataset

/bin/bash: listdir: command not found


In [64]:
import os
folders=os.listdir("./drive/My Drive/Images")
print(folders)

['Humans', 'cats', 'dogs', 'horses']


In [0]:
image_data=[]
labels=[]

label_dict={
    
    "Humans":0,
    "cats":1,
     "dogs":2,
     "horses":3,
}

In [87]:
for ix in folders:
  path=os.path.join("./drive/My Drive/Images",ix)
  print(path)
  for im in os.listdir(path):
    img=image.load_img(os.path.join(path,im),target_size=(224,224))
    img_array=image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])
    

  
  

./drive/My Drive/Images/Humans
./drive/My Drive/Images/cats
./drive/My Drive/Images/dogs
./drive/My Drive/Images/horses


In [0]:
from keras.preprocessing import image

In [89]:
print(len(image_data))
print(len(labels))

808
808


In [0]:
import random
combined=list(zip(image_data,labels))
random.shuffle(combined)

image_data[:],labels[:]=zip(*combined)

In [94]:
print(labels[:5])

[2, 0, 2, 1, 1]


In [95]:
import numpy as np
X_train=np.array(image_data)
Y_train=np.array(labels)
print(X_train.shape)
print(Y_train.shape)

(808, 224, 224, 3)
(808,)


In [96]:
from keras.utils import np_utils
Y_train=np_utils.to_categorical(Y_train)
print(X_train.shape)
print(Y_train.shape)

(808, 224, 224, 3)
(808, 4)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import *
from matplotlib import pyplot as plt

In [103]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [104]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [107]:
from keras.layers.pooling import GlobalAveragePooling2D
av1=GlobalAveragePooling2D()(model.output)
fc1= Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2= Dense(4,activation='softmax')(d1)

model_new=Model(input=model.input,outputs=fc2)
model_new.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  import sys


In [0]:
adam=Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [109]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7fb30c348390>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7fb30caad128>
2 <keras.layers.convolutional.Conv2D object at 0x7fb30c3483c8>
3 <keras.layers.normalization.BatchNormalization object at 0x7fb30c236438>
4 <keras.layers.core.Activation object at 0x7fb30c271240>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7fb30c3fae10>
6 <keras.layers.pooling.MaxPooling2D object at 0x7fb30c4dc438>
7 <keras.layers.convolutional.Conv2D object at 0x7fb30c3aeef0>
8 <keras.layers.normalization.BatchNormalization object at 0x7fb30c0f84e0>
9 <keras.layers.core.Activation object at 0x7fb30c0f8e80>
10 <keras.layers.convolutional.Conv2D object at 0x7fb30c0a3d68>
11 <keras.layers.normalization.BatchNormalization object at 0x7fb30c054b00>
12 <keras.layers.core.Activation object at 0x7fb30c078a20>
13 <keras.layers.convolutional.Conv2D object at 0x7fb30bf9e048>
14 <keras.layers.convolutional.Conv2D object at 0x7fb30bf73390>
15 <keras.lay

In [112]:
for ix in range(169):
  model_new.layers[ix].trainable=False
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_new.summary()
 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [114]:
hist=model_new.fit(X_train,Y_train,epochs=5,shuffle=True,batch_size=16,validation_split=0.20)

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 239s 369ms/step - loss: 0.2104 - acc: 0.9396 - val_loss: 0.1434 - val_acc: 0.9506
Epoch 2/5
646/646 [==============================] - 235s 364ms/step - loss: 0.1641 - acc: 0.9505 - val_loss: 0.1101 - val_acc: 0.9568
Epoch 3/5
646/646 [==============================] - 236s 366ms/step - loss: 0.1546 - acc: 0.9598 - val_loss: 0.1201 - val_acc: 0.9568
Epoch 4/5
646/646 [==============================] - 242s 374ms/step - loss: 0.1479 - acc: 0.9582 - val_loss: 0.1150 - val_acc: 0.9568
Epoch 5/5
646/646 [==============================] - 243s 376ms/step - loss: 0.1225 - acc: 0.9644 - val_loss: 0.1314 - val_acc: 0.9506


**Thus we get an acuuracy of 96% on training data and 95% on validation data.!**